# Links featurized lines to their verdicts

We need this to add the verdict to the lines that have been featurized

In [1]:
import pandas as pd

This code reads the csv file and creates a dictionary for the lines, which we can later on use when adding a verdict to a specific line. 

In [5]:
dfLines = pd.read_csv('./gerrit_reviews_comments_dictionary_sentiment_cleaned.csv', 
                      sep='$',
                     error_bad_lines=False,
                     warn_bad_lines=True,
                     index_col=False)

In [6]:
dfLines.head()

,filename,LOC,class_value
0,src/gromacs/applied_forces/densityfitting.cpp,"basic_mdspan<const float, dynamicExten...",0
1,src/gromacs/applied_forces/densityfitting.cpp,"basic_mdspan<const float, dynamicExten...",1
2,src/gromacs/applied_forces/densityfitting.cpp,"* Asserts that reading went okay, exi...",0
3,src/gromacs/applied_forces/densityfitting.cpp,"* Asserts that reading went okay, exi...",1
4,src/gromacs/applied_forces/densityfitting.cpp,= std::make_unique < Translate...,0


In [8]:
dictLines = dict()

for index, row in dfLines.iterrows():
    line = row['LOC']
    verdict_in_csv = row['class_value']
    if line in dictLines:
        verdict_in_dict = dictLines[line]
        if verdict_in_csv != verdict_in_dict:
            dictLines[line] = 1
    else:
        dictLines[line] = verdict_in_csv
        

In [9]:
dfLinesVerdicts = pd.DataFrame.from_dict(dictLines, orient='index', columns=['class_value'])

In [10]:
dfLinesVerdicts.index.name = 'line'
dfLinesVerdicts.head()

,class_value
line,
"basic_mdspan<const float, dynamicExtents3D> referenceDensity() const",1
"* Asserts that reading went okay, exits with fatal error otherwise.",1
= std::make_unique < TranslateAndScale>(reader.transformationToDensityLattice())_,1
"const std::string c_referenceDensityFileNameTag_ = ""reference-density-filename""_",1
DensityFittingParameters parameters__,1


In [11]:
# uncomment this line if you want to use a pre-created dictionary with verdicts
#dfLinesVerdicts.to_csv('./gerrit_comments_dictionary_cleaned.csv')

In [12]:
dfLinesFeaturized = pd.read_csv('./featurized_lines.csv', 
                                sep='$', 
                               encoding='utf8',
                               error_bad_lines = False, 
                               index_col=0)

In [13]:
dfLinesFeaturized['class_value'] = -1

In [14]:
for iLine, oneLine in dfLinesFeaturized.iterrows():
    if iLine in dictLines:
        verdict = dictLines[iLine]
    else:
        verdict = 1
    #print(iLine)
    dfLinesFeaturized.loc[iLine, 'class_value'] = verdict

In [15]:
dfLinesFeaturized.to_csv('./train_lines.csv', sep='$', na_rep='0')